In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from scipy import ndimage
from allfunc import *
import random
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd

In [2]:
def scale_factor_spectrum(z,data,error,wo_wave):
    k = 1+z
    select1 = (wave>6750*k) & (wave<6800*k)
    cont_dat = data[select1]
    cont_err = error[select1]
    error_maincube = np.std(cont_dat)
    error_errcube = np.mean(cont_err)
    fact = error_maincube/error_errcube
    return fact

def create_3_arcsec_minicube(wo_cube,wo_err,wo_header,cont_cube,brightest_pixel_x,brightest_pixel_y,sampling_rate,box_size=3):
    if sampling_rate == 0.4:
        mini_cube_data = wo_cube[:,brightest_pixel_y-box_size:brightest_pixel_y+box_size+1,brightest_pixel_x-box_size:brightest_pixel_x+box_size+1]
        mini_cube_err = wo_err[:,brightest_pixel_y-box_size:brightest_pixel_y+box_size+1,brightest_pixel_x-box_size:brightest_pixel_x+box_size+1]
    else:
        mini_cube_data = wo_cube[:,brightest_pixel_y-2*box_size:brightest_pixel_y+2*box_size+1,brightest_pixel_x-2*box_size:brightest_pixel_x+2*box_size+1]
        mini_cube_err = wo_err[:,brightest_pixel_y-2*box_size:brightest_pixel_y+2*box_size+1,brightest_pixel_x-2*box_size:brightest_pixel_x+2*box_size+1]
    wo_header['CRPIX1'] = wo_header['CRPIX1'] - (brightest_pixel_x-box_size)
    wo_header['CRPIX2'] = wo_header['CRPIX2'] - (brightest_pixel_y-box_size)
    return mini_cube_data, mini_cube_err,wo_header

def emp_fact(z,mini_cube,mini_err,wo_wave):
    true_err = np.zeros((np.shape(wo_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    for i in range(mini_cube.shape[1]):
        for j in range(mini_cube.shape[2]):
            k = 1+z
            select1 = (wo_wave>6750*k) & (wo_wave<6850*k)
   
            (spectrum,error) = (mini_cube[:,i,j],mini_err[:,i,j])  
            (spec,err) = (spectrum[select1],error[select1])
            (error_maincube,error_errcube) = (np.std(spec),np.mean(err))
            fact = error_maincube/error_errcube
            new_err = fact*error
            true_err[:,i,j] = new_err 
    return true_err

In [3]:
def mini_dat_and_err(x1,y1,qso_data,qso_error,asymmetry=False,box_size=4):
    if asymmetry:
        mini_data = qso_data[:,y1:y1+box_size,x1:x1+box_size]
        mini_err = qso_error[:,y1:y1+box_size,x1:x1+box_size]
    else:
        mini_data = qso_data[:,y1:y1+box_size+1,x1:x1+box_size+1]
        mini_err = qso_error[:,y1:y1+box_size+1,x1:x1+box_size+1]
    return mini_data,mini_err

def emission_cont_dat(z,qso_wave,mini_data,mini_error):
    k = 1+z
    select_emission = (qso_wave>4990*k) & (qso_wave<5017*k)
    select_cont = (qso_wave>5030*k) & (qso_wave<5050*k)
    cont_wave = qso_wave[select_cont]
    cont_data = mini_data[select_cont]
    emission_wave = qso_wave[select_emission]
    emission_data = mini_data[select_emission]
    return cont_data,emission_data

def noise(mini_data,cont_data):
    noise = np.zeros((mini_data.shape[1],mini_data.shape[2]))
    for i in range(mini_data.shape[1]):
        for j in range(mini_data.shape[2]):
            noise[i,j]= np.std(cont_data[:,i,j])
    N = noise
    return N

def signal(mini_data,emission_data):
    S = np.sum(np.sum(emission_data[:,i,j] for i in range(mini_data.shape[1]) for j in range(mini_data.shape[2])))
    return S

def no_of_pix(mini_data):
    number = mini_data.shape[1]*mini_data.shape[2]
    return number

def cordinate_tab(obj,m,x_act,y_act,x1,y1,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    full_data = [x_act,y_act,m]
    column_names={'x_cor':0,'y_cor':1,'No_iteration':2}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[full_data[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s/%s/Systematic Error/x%sy%s_artificial_cordinatecentertab_%s.fits'%(destination_path_cube,obj,x1,y1,obj),overwrite=True)   

In [4]:
def algorithm_script(obj,z,x1,y1,prefix_path_cube="/home/mainak/ftp.hidrive.strato.com/users/login-carsftp/IFU_data",destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    #print '%s'%(obj)    
    (qso_data,qso_error,qso_wave,qso_header) = loadCube('%s/%s/%s.wo_absorption.fits'%(destination_path_cube,obj,obj)) 
    (cont_cube,cont_err,cont_wave,cont_header) = loadCube('%s/MUSE/%s/fitting/full/%s.cont_model.fits'%(prefix_path_cube,obj,obj))
    sampling_rate = sampling_size(cont_cube)
    
    (mini_data,mini_err,wo_header)=create_3_arcsec_minicube(qso_data,qso_error,qso_header,cont_cube,x1,y1,sampling_rate,box_size=3)
    true_err = emp_fact(z,mini_data,mini_err,qso_wave)
    mini_error = true_err
    
    #print x1,y1
    #(mini_data,mini_error) = mini_dat_and_err(x1,y1,qso_data,qso_error,asymmetry,box_size=7)
    (cont_data,emission_data) = emission_cont_dat(z,qso_wave,mini_data,mini_error)
    
    N = noise(mini_data,cont_data)
    S = signal(mini_data,emission_data)
    pix_no = no_of_pix(mini_data)
    S_N_ratio = np.divide(S,N*pix_no)
    
    select_ratio = (S_N_ratio>-3) & (S_N_ratio<3)
    ratio = S_N_ratio[select_ratio] 
    (s,) = np.shape(ratio)
    if s == pix_no:         # it means it is a 'good source' since S/N in all spaxel is -5 < S/N < 5
        (x_act,y_act) = (x1,y1)
        #print (x_act,y_act) 
        cordinate_tab(obj,m,x_act,y_act,x1,y1)  
    else:
        pass              # else we are simply avoiding the generated coordinates
   

In [6]:
z = {"HE1107-0813":0.058}

objs = z.keys()

for obj in objs:
    print ('%s'%(obj))
    for m in range(10000):
        (x_min,y_min,x_max,y_max) = shape(obj)
        (x1,y1) = (np.random.randint(x_min,x_max),np.random.randint(y_min,y_max))
        algorithm_script(obj,z[obj],x1,y1) 
        filename='./*cordinatecentertab_%s.fits'%(obj)
        destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"
        dir_name = '%s/%s/Systematic Error'%(destination_path_cube,obj)   
        os.chdir(dir_name)
        infile = sorted(glob.glob(filename))
        #print (len(infile))
        if len(infile) == 300:
            destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"
            input_filename='./*cordinatecentertab_%s.fits'%(obj)
            output_filename='%s_cord_table.fits'%(obj)

            dir_name = '%s/%s/Systematic Error'%(destination_path_cube,obj)   
            os.chdir(dir_name)
            infiles = sorted(glob.glob(input_filename)) # './*source.fits'
            tabledict = {}
    
            for i, file in enumerate(infiles):
                hdulist = fits.open(file)
                bintab = hdulist[1].data
                table = Table(bintab)
                name = table['No_iteration'][0]  
                tabledict[name] = table

            tab = vstack(list(tabledict.values()))
            df = tab.to_pandas()
            df = df.sort_values(by='No_iteration', ascending=True)
            dtab = Table.from_pandas(df)
            dtab.write(output_filename, format='fits', overwrite=True)
            for i, file in enumerate(infiles):
                   os.remove(file)
            break
        else:
            continue

HE1107-0813


/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


In [5]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

for obj in objs:
    print ('%s'%(obj))
    for m in range(10000):
        (x_min,y_min,x_max,y_max) = shape(obj)
        (x1,y1) = (np.random.randint(x_min,x_max),np.random.randint(y_min,y_max))
        algorithm_script(obj,z[obj],x1,y1) 
        filename='./*cordinatecentertab_%s.fits'%(obj)
        destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"
        dir_name = '%s/%s/Systematic Error'%(destination_path_cube,obj)   
        os.chdir(dir_name)
        infile = sorted(glob.glob(filename))
        #print (len(infile))
        if len(infile) == 300:
            destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"
            input_filename='./*cordinatecentertab_%s.fits'%(obj)
            output_filename='%s_cord_table.fits'%(obj)

            dir_name = '%s/%s/Systematic Error'%(destination_path_cube,obj)   
            os.chdir(dir_name)
            infiles = sorted(glob.glob(input_filename)) # './*source.fits'
            tabledict = {}
    
            for i, file in enumerate(infiles):
                hdulist = fits.open(file)
                bintab = hdulist[1].data
                table = Table(bintab)
                name = table['No_iteration'][0]  
                tabledict[name] = table

            tab = vstack(list(tabledict.values()))
            df = tab.to_pandas()
            df = df.sort_values(by='No_iteration', ascending=True)
            dtab = Table.from_pandas(df)
            dtab.write(output_filename, format='fits', overwrite=True)
            for i, file in enumerate(infiles):
                   os.remove(file)
            break
        else:
            continue

HE0021-1810


/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in true_divide


HE0021-1819
HE0040-1105
HE0108-4743
HE0114-0015
HE0119-0118


/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in greater
/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


HE0212-0059
HE0224-2834
HE0227-0913
HE0232-0900
HE0253-1641
HE0345+0056
HE0351+0240
HE0412-0803
HE0429-0247
HE0433-1028
HE0853+0102
HE0934+0119
HE1011-0403
HE1017-0305
HE1029-1831
HE1107-0813
HE1108-2813
HE1126-0407
HE1237-0504
HE1248-1356
HE1330-1013
HE1353-1917
HE1417-0909
HE2128-0221
HE2211-3903
HE2222-0026
HE2233+0124
HE2302-0857
